In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import multiprocessing
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import LassoCV

In [2]:
def load_dataset(path):
    dataset = pd.read_csv(path)
    dataset = dataset.sample(frac=1).reset_index(drop=True).to_numpy()
    return dataset

In [3]:
def normal_dataset(dataset):
    dataset = preprocessing.normalize(dataset)
    return dataset

In [4]:
def create_train_test_data(dataset):
    index = int(0.8 * len(dataset))
    train = dataset[:index]
    test = dataset[index:]
    features, b1, b2 = train[:,:8], train[:,8], train[:,9]
    features_test, b1_test, b2_test = test[:,:8], test[:,8], test[:,9]
    b1 = b1.reshape((-1,1))
    b2 = b2.reshape((-1,1))
    b1_test = b1_test.reshape((-1,1))
    b2_test = b2_test.reshape((-1,1))
    return features, b1, b2,features_test, b1_test, b2_test


In [5]:
def calculte_x_t(A,b,ro,z,nu,d):
    t1 = np.linalg.inv(A.T.dot(A) + ro * np.identity(d))
    t2 = A.T.dot(b) + ro*z -  nu
    return t1.dot(t2)

In [6]:
def calculte_z_t(z, x, nu, ro, landa):
    e_grad = x + nu*1.0/ro
    z_t = np.zeros_like(z)
    l = -(1.0*landa/ro)*(z<0)
    g = (1.0*landa/ro)*(z>0)
    z_t = e_grad - l - g
    return z_t

In [7]:
def calculate_nu_t(nu_t,ro,X_t,z_t):
    nu_t = nu_t + ro*(X_t - z_t)
    return nu_t

In [8]:
def calculate_value(A,X,b,landa):
    value = 0.5*np.linalg.norm(A.dot(X) - b, ord='fro')**2 + landa*np.linalg.norm(X, ord=1)
    return value

In [19]:
def print_status(status,A,A_test,X_t,b,b_test,landa,iteration):
    if status == 0:
        print("First value:",calculate_value(A,X_t,b,landa))
        print("--------------------------------")
    elif status == 1:
        print("--------------------------------")
        print("Iteration " + str(iteration+1))
        print("value:",calculate_value(A,X_t,b,landa))
        print("Rmse of Train Data: "+str(mean_squared_error(A.dot(X_t),b)))
        print("Rmse of Test Data: "+str(mean_squared_error(A_test.dot(X_t),b_test)))
        print("--------------------------------")
    else:
        print("--------------------------------")
        print("--------Final Results--------")
        print("Final value:",calculate_value(A,X_t,b,landa))
        print("Rmse of Train Data: "+str(mean_squared_error(A.dot(X_t),b)))
        print("Rmse of Test Data: "+str(mean_squared_error(A_test.dot(X_t),b_test)))
        print("--------------------------------")


In [9]:
def LassoCV_Regresion(A,b,A_test,b_test):
    model = LassoCV(cv=5, random_state=0).fit(A, b)
    model.score(A, b)
    predict_train = model.predict(A)
    predict_test = model.predict(A_test)
    
    print("--------LassoCV Results--------")
    print("Rmse of Train Data: "+str(mean_squared_error(predict_train,b)))
    print("Rmse of Test Data: = "+str(mean_squared_error(predict_test,b_test)))
    print("--------------------------------")

In [21]:
def none_Parallel_ADMM(A,b,A_test , b_test,d,iterations,landa):
    n = len(A)
    X_t = np.random.randn(d, 1)
    z_t = np.zeros((d,1))
    ro = 1
    nu_t = np.zeros((d,1))
    print_status(0,A,A_test,X_t,b,b_test,landa,0)
    for iteration in range(iterations):
        X_t = calculte_x_t(A,b,ro,z_t,nu_t,d)
        z_t = calculte_z_t(z_t, X_t, nu_t, ro, landa)
        nu_t = calculate_nu_t(nu_t,ro,X_t,z_t)
        print_status(1,A,A_test,X_t,b,b_test,landa,iteration)
    print_status(2,A,A_test,X_t,b,b_test,landa,iterations)

In [ ]:
def calculate_pararell(i,A,b,nu_t,ro,z_t,X_t):
    ai = A[i].reshape(-1,1).T
    bi = b[i].reshape(-1,1)
    bi = bi.item()
    nui = nu_t[i].reshape(-1,1)
    term1 = 1/(ai.dot(ai.T) + ro)
    term2 = ai.T*bi + ro*z_t - nui
    term1 = term1.item()
    x = term1*term2
    x = x.T
    X_t[i] = x

In [ ]:
def update_nu_t_pararell(nu_t,X_t,z_t,ro):
    for i in range(nu_t.shape[0]):
        nu_t[i] = (nu_t[i].reshape(-1,1) + ro*(X_t[i].reshape(-1,1) - z_t)).T
    return nu_t

In [27]:
def Parallel_ADMM(A,b,A_test , b_test,d,iterations,landa):
    n = len(A)
    X_t = np.tile(np.random.randn(d, 1).T, (n,1))
    z_t = np.zeros((d,1))
    ro = 1
    nu_t = np.tile(np.zeros((d,1)).T, (n,1))

    X = np.mean(X_t, axis=0).reshape(-1,1)
    print_status(0,A,A_test,X,b,b_test,landa,0)

    for iteration in range(iterations):
        Parallel(n_jobs=multiprocessing.cpu_count() , require='sharedmem')(delayed(calculate_pararell)(i,A,b,nu_t,ro,z_t,X_t) for i in range(n))
        X = np.mean(X_t, axis=0).reshape(-1,1)
        nu = np.mean(nu_t, axis=0).reshape(-1,1)
        z_t = calculte_z_t(z_t, X, nu, ro, landa)
        nu_t = update_nu_t_pararell(nu_t,X_t,z_t,ro)
        print_status(1,A,A_test,X,b,b_test,landa,iteration)
    print_status(2,A,A_test,X,b,b_test,landa,iterations)


In [12]:
dataset = load_dataset("ENB2012_data.csv")
dataset_normal = normal_dataset(dataset)
features, b1, b2,features_test, b1_test, b2_test = create_train_test_data(dataset_normal)
d=8

In [22]:
none_Parallel_ADMM(features,b1,features_test , b1_test,d,100,0.01)

First value: 32.447879433381075
--------------------------------
--------------------------------
Iteration 1
value: 0.023565089081114616
Rmse of Train Data: 6.982374875716604e-05
Rmse of Test Data: 7.232542115535657e-05
--------------------------------
--------------------------------
Iteration 2
value: 0.02166286317584736
Rmse of Train Data: 6.174295529756527e-05
Rmse of Test Data: 6.403915862816915e-05
--------------------------------
--------------------------------
Iteration 3
value: 0.02222495696522428
Rmse of Train Data: 6.234288524409262e-05
Rmse of Test Data: 6.474869085748859e-05
--------------------------------
--------------------------------
Iteration 4
value: 0.02202491126334219
Rmse of Train Data: 6.162065638081358e-05
Rmse of Test Data: 6.389497480593596e-05
--------------------------------
--------------------------------
Iteration 5
value: 0.022145327708546363
Rmse of Train Data: 6.210456234451195e-05
Rmse of Test Data: 6.44242198385621e-05
---------------------------

In [14]:
none_Parallel_ADMM(features,b2,features_test , b2_test,d,100,0.01)

First value: 374.16920910981105
--------------------------------
--------------------------------
Iteration 1
value: 0.024349405311560293
Rmse of Train Data: 7.272753263795677e-05
Rmse of Test Data: 6.435581187758333e-05
--------------------------------
--------------------------------
Iteration 2
value: 0.02262008220627245
Rmse of Train Data: 6.526116010358512e-05
Rmse of Test Data: 5.762531713718427e-05
--------------------------------
--------------------------------
Iteration 3
value: 0.023202611500030994
Rmse of Train Data: 6.589171309265758e-05
Rmse of Test Data: 5.831552968792386e-05
--------------------------------
--------------------------------
Iteration 4
value: 0.022976570328344492
Rmse of Train Data: 6.519437918154841e-05
Rmse of Test Data: 5.762636134685703e-05
--------------------------------
--------------------------------
Iteration 5
value: 0.023085561762230575
Rmse of Train Data: 6.56625092930721e-05
Rmse of Test Data: 5.812256989206721e-05
-------------------------

In [28]:
Parallel_ADMM(features,b1,features_test , b1_test,d,100,0.01)

First value: 557.7358153940053
--------------------------------
--------------------------------
Iteration 1
value: 0.1468965152601888
Rmse of Train Data: 0.000477723323172123
Rmse of Test Data: 0.00045193386380386677
--------------------------------
--------------------------------
Iteration 2
value: 0.09298949157292215
Rmse of Train Data: 0.00030174665092560403
Rmse of Test Data: 0.0002877470163525489
--------------------------------
--------------------------------
Iteration 3
value: 0.19050839142051254
Rmse of Train Data: 0.0006182996053043145
Rmse of Test Data: 0.0005863101514938365
--------------------------------
--------------------------------
Iteration 4
value: 0.1302031358154435
Rmse of Train Data: 0.00042137269702800535
Rmse of Test Data: 0.000399252612260283
--------------------------------
--------------------------------
Iteration 5
value: 0.15919907337554756
Rmse of Train Data: 0.0005162147252677312
Rmse of Test Data: 0.0004886320566113676
------------------------------

In [16]:
Parallel_ADMM(features,b2,features_test , b2_test,d,100,0.01)

Initial value: 88.77033700576831
--------------------------------
Iteration 1
value: 0.15873357637482374
Rmse of Train Data: 0.0005162046009716508
Rmse of Test Data: = 0.0004666442754517424
--------------------------------
--------------------------------
Iteration 2
value: 0.09361900107472096
Rmse of Train Data: 0.00030368323148786453
Rmse of Test Data: = 0.0002704465046740705
--------------------------------
--------------------------------
Iteration 3
value: 0.19200893322246498
Rmse of Train Data: 0.000623097940276678
Rmse of Test Data: = 0.0005678881665299293
--------------------------------
--------------------------------
Iteration 4
value: 0.12831815937743637
Rmse of Train Data: 0.0004150902065776575
Rmse of Test Data: = 0.000372631791610482
--------------------------------
--------------------------------
Iteration 5
value: 0.15709612322039812
Rmse of Train Data: 0.000509265685474802
Rmse of Test Data: = 0.0004601637519115591
--------------------------------
-------------------

In [17]:
LassoCV_Regresion(features,b1,features_test , b1_test)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


--------LassoCV Results--------
Rmse of Train Data: 3.183976894072879e-05
Rmse of Test Data: = 3.1023796351080806e-05
--------------------------------


In [18]:
LassoCV_Regresion(features,b2,features_test , b2_test)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


--------LassoCV Results--------
Rmse of Train Data: 3.0187622468237413e-05
Rmse of Test Data: = 2.24480327060567e-05
--------------------------------
